In [16]:
import math as m

In [29]:
import numpy as np

In [ ]:
%pylab
%matplotlib inline

In [19]:
def TESinSol(f,t0,u0,imax,h): #Trapeció Explícito para cuando no sabes la sol exacta 
    w = np.zeros(imax+1)
    t = np.zeros(imax+1)
    e = np.zeros(imax+1) #Error local
    w[0] = u0
    t[0] = t0
    for i in range(0,imax):
        t[i+1] = t[i] + h
        w[i+1] = w[i] + 0.5*h*(f(t[i],w[i]) + f(t[i]+h, w[i] + h*f(t[i],w[i])))
    return (t,w,e)

In [14]:
def pasoFlexible (f, wj, tj, tmax, tol):
    check = False
    trel = tol * max(abs(wj), 0.25)
    H = tmax - tj
    while (check == False):
        E = abs(RK4SinSol(f, tj, wj, imax, H)[1][len(RK4SinSol(f, tj, wj, imax, H)[1])-1]-TESinSol(f, tj, wj, imax, H)[1][len(TESinSol(f, tj, wj, imax, H)[1])-1])
        if E<tol:
            check = True
        else:
            H = H/2
    tf = tj + H
    wf = RK4SinSol(f, tj, wj, imax, H)[1][len(RK4SinSol(f, tj, wj, imax, H)[1])-1]
    Hsug = 2*H
    return tf, wf, Hsug

In [15]:
def pasoFlexibleEficiente (f, wj, tj, tmax, tol):
    trel = tol * max(abs(wj), 0.25)
    H = tmax - tj
    E = abs(RK4SinSol(f, tj, wj, imax, H)[1][len(RK4SinSol(f, tj, wj, imax, H)[1])-1]-TESinSol(f, tj, wj, imax, H)[1][len(TESinSol(f, tj, wj, imax, H)[1])-1])
    while (E>=tol):
        H = H/2
        E = abs(RK4SinSol(f, tj, wj, imax, H)[1][len(RK4SinSol(f, tj, wj, imax, H)[1])-1]-TESinSol(f, tj, wj, imax, H)[1][len(TESinSol(f, tj, wj, imax, H)[1])-1])
    tf = tj + H
    wf = RK4SinSol(f, tj, wj, imax, H)[1][len(RK4SinSol(f, tj, wj, imax, H)[1])-1]
    Hsug = 2*H
    return tf, wf, Hsug

In [17]:
def metodoAdaptativo (f, u0, T, Hmax, tol):
    numpasos = m.ceil(abs(T[0]-T[1])/Hmax)
    aprox = [[0 for i in range(2)] for j in range(numpasos+1) ]
    Hsug = Hmax
    tp = T[0]
    wp = u0
    tmax = min(T[1], tp+Hsug)
    [tf, wf, Hsug] = pasoFlexibleEficiente(f, wp, tp, tmax, tol)
    Hsug = min (Hmax, Hsug)
    aprox[0][0] = tf
    aprox[0][1] = wf
    i = 1
    while (tf!= T[1]):
        tp = tf
        wp = wf
        tmax = min(T[1], tp+Hsug)
        [tf, wf, Hsug] = pasoFlexibleEficiente(f, wp, tp, tmax, tol)
        Hsug = min (Hmax, Hsug)
        aprox[i][0] = tf
        aprox[i][1] = wf
        i = i+1
    return aprox

In [45]:
def RK4SinSol(f,t0,u0,imax,h):
    w = [[0 for i in range(2)] for j in range(imax+1) ]
    w[0][0]= u0
    w[0][1] = u0
    t = np.zeros(imax+1) 
    t[0] = t0
    e = np.zeros(imax+1)
    for i in range(0,imax):
        s1 = f(t[i], w[i][0], w[i][1])
        s2 = f(t[i]+h/2, w[i][0]+(h/2)*s1, w[i][1]+(h/2)*s1)
        s3 = f(t[i]+h/2, w[i][0]+(h/2)*s2, w[i][1]+(h/2)*s2)
        s4 = f(t[i]+h, w[i][0]+h*s3, w[i][1]+h*s3)
        w[i+1][0] = w[i][0]+(h/6)*(s1+2*s2+2*s3+s4)
        w[i+1][1] = w[i][1]+(h/6)*(s1+2*s2+2*s3+s4)
        t[i+1] = t[i]+h
    w1 = w[imax]
    return (t,w1,e)

In [44]:
def f1 (u2, y1, y2):
    m = 0.012277471
    return y1 + 2*u2 - (1-m)*((y1+m)/(((y1+m)**2)+y2**2)**(3/2)) - m*((y1-(1-m))/((((y1-(1-m))**2)+y2**2)**(3/2)))

In [43]:
def f2 (u1, y1, y2):
    m = 0.012277471
    return y2-2*u1-(1-m)*(y2/((((y1+m)**2)+y2**2)**(3/2))) - m*(y2/((((y1-(1-m))**2)+y2**2)**(3/2)))

In [46]:
def f3 (u1, y1, y2):
    return u1

In [47]:
def f4 (u1, y1, y2):
    return u2

In [48]:
def f (t, u1, u2, y1, y2):
    return f1(u2, y1, y2), f2(u1, y1, y2), f3(u1), f4(t, u1, u2, y1, y2)

In [ ]:
metodoAdaptativo(f)

In [53]:
def TrapExpVariable(f, f1, f2, f3, f4, t0,v1_0,v2_0,y1_0,y2_0,imax,h0,tol):
    i = 0
    h = h0
    w_v1 = np.zeros(imax+1)
    w_v2 = np.zeros(imax+1)
    w_y1 = np.zeros(imax+1)
    w_y2 = np.zeros(imax+1)
    t = np.zeros(imax+1)
    t[0] = t0
    w_v1[0] = v1_0
    w_v2[0] = v2_0
    w_y1[0] = y1_0
    w_y2[0] = y2_0
    while ((abs(any(RK4SinSol(f1,t0,v1_0,i,h)[1])-w_v1[i]) >= tol) or (abs(any(RK4SinSol(f2,t0,v2_0,i,h)[1])-w_v2[i]) >= tol) or (abs(any(RK4SinSol(f3,t0,y1_0,i,h)[1])-w_y1[i]) >= tol) or (abs(any(RK4SinSol(f4,t0,y2_0,i,h)[1])-w_y2[i]) >= tol) or (i <= imax)):
        w_v1[i+1] = w_v1[i] + (h/2)*(f(t[i], w_v1[i], w_v2[i], w_y1[i], w_y2[i])[0] + f(t[i]+h, w_v1[i] + h*f(t[i],w_v1[i],w_v2[i],w_y1[i],w_y2[i])[0], w_v2[i] + h*f(t[i],w_v1[i],w_v2[i],w_y1[i],w_y2[i])[1], w_y1[i] + h*f(t[i],w_v1[i],w_v2[i],w_y1[i],w_y2[i])[2], w_v1[i] + h*f(t[i],w_v1[i],w_v2[i],w_y1[i],w_y2[i])[3])[0])
        w_v2[i+1] = w_v2[i] + (h/2)*(f(t[i], w_v1[i], w_v2[i], w_y1[i], w_y2[i])[1] + f(t[i]+h, w_v1[i] + h*f(t[i],w_v1[i],w_v2[i],w_y1[i],w_y2[i])[0], w_v2[i] + h*f(t[i],w_v1[i],w_v2[i],w_y1[i],w_y2[i])[1], w_y1[i] + h*f(t[i],w_v1[i],w_v2[i],w_y1[i],w_y2[i])[2], w_v1[i] + h*f(t[i],w_v1[i],w_v2[i],w_y1[i],w_y2[i])[3])[1])
        w_y1[i+1] = w_y1[i] + (h/2)*(f(t[i], w_v1[i], w_v2[i], w_y1[i], w_y2[i])[2] + f(t[i]+h, w_v1[i] + h*f(t[i],w_v1[i],w_v2[i],w_y1[i],w_y2[i])[0], w_v2[i] + h*f(t[i],w_v1[i],w_v2[i],w_y1[i],w_y2[i])[1], w_y1[i] + h*f(t[i],w_v1[i],w_v2[i],w_y1[i],w_y2[i])[2], w_v1[i] + h*f(t[i],w_v1[i],w_v2[i],w_y1[i],w_y2[i])[3])[2])
        w_y2[i+1] = w_y2[i] + (h/2)*(f(t[i], w_v1[i], w_v2[i], w_y1[i], w_y2[i])[3] + f(t[i]+h, w_v1[i] + h*f(t[i],w_v1[i],w_v2[i],w_y1[i],w_y2[i])[0], w_v2[i] + h*f(t[i],w_v1[i],w_v2[i],w_y1[i],w_y2[i])[1], w_y1[i] + h*f(t[i],w_v1[i],w_v2[i],w_y1[i],w_y2[i])[2], w_v1[i] + h*f(t[i],w_v1[i],w_v2[i],w_y1[i],w_y2[i])[3])[3])
        t[i+1] = t[i] + h 
        h = h/2
        i = i+1
    return (t,w_v1,w_v2,w_y1,w_y2)

In [54]:
TrapExpVariable(f, f1, f2, f3, f4, 0, 0, -2.00158510637908252240537862224, 0.994, 0, 10, 1, 0.001)

TypeError: f1() takes 3 positional arguments but 5 were given